In [1]:
import gc
import time
import numpy as np
import pandas as pd
#from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
%matplotlib inline

In [2]:
train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

In [3]:
#train = pd.read_csv("train_sample.csv", usecols=train_columns, dtype=dtypes)#, nrows=6000)
#train = pd.read_csv("train.csv", skiprows=range(1,123903891), nrows=61000000, usecols=train_columns, dtype=dtypes)#, nrows=6000)
train = pd.read_csv("train.csv", skiprows=range(1,175903891), nrows=5000000, usecols=train_columns, dtype=dtypes)#, nrows=6000)
#train.head()

In [4]:
test = pd.read_csv("test.csv", dtype=dtypes, usecols=test_columns)
#test = pd.read_csv("test.csv", dtype=dtypes, usecols=test_columns, nrows=6000)
#test = pd.read_csv("test_supplement.csv", dtype=dtypes, usecols=test_columns)
test.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [5]:
y = train['is_attributed']
train.drop(['is_attributed'], axis=1, inplace=True)
train.head()

,ip,app,device,os,channel,click_time
0,44725,9,1,37,445,2017-11-09 13:16:16
1,100186,12,1,19,205,2017-11-09 13:16:16
2,90485,12,1,13,259,2017-11-09 13:16:16
3,73100,9,1,13,232,2017-11-09 13:16:16
4,5314,13,1,13,469,2017-11-09 13:16:16


### Считаем количество кликов по ip

In [6]:
#nrow_train = train.shape[0]
#merge = pd.concat([train, test])

In [7]:
#del train, test
#gc.collect()

In [8]:
# Count the number of clicks by ip
#ip_count = merge.groupby(['ip'])['channel'].count().reset_index()
#ip_count.columns = ['ip', 'clicks_by_ip']
#merge = pd.merge(merge, ip_count, on='ip', how='left', sort=False)
#merge['clicks_by_ip'] = merge['clicks_by_ip'].astype('uint16')
##merge.drop('ip', axis=1, inplace=True)

In [9]:
#train = merge[:nrow_train]
#test = merge[nrow_train:]

In [10]:
##del test, merge
#del merge
#gc.collect()

### Из click_time генерим фичи

In [11]:
def timeFeatures(train):
    #time
    #train["doy"]     = pd.to_datetime(train.click_time).dt.dayofyear.astype('uint8')
    train['wday']    = pd.to_datetime(train.click_time).dt.dayofweek#.astype('uint8')
    train['day']     = pd.to_datetime(train.click_time).dt.day#.astype('uint8')
    train['hour']    = pd.to_datetime(train.click_time).dt.hour#.astype('uint8')
    train['minute']  = pd.to_datetime(train.click_time).dt.minute#.astype('uint8')
    train['second']  = pd.to_datetime(train.click_time).dt.second#.astype('uint8')
    train.drop(['click_time'], axis=1, inplace=True)
    return train

#print(train.dtypes) 

In [12]:
train = timeFeatures(train)
gc.collect()

53

### Проверяем train и test df

In [13]:
#train.drop(['click_id'], axis=1, inplace=True)
train.head()

,ip,app,device,os,channel,wday,day,hour,minute,second
0,44725,9,1,37,445,3,9,13,16,16
1,100186,12,1,19,205,3,9,13,16,16
2,90485,12,1,13,259,3,9,13,16,16
3,73100,9,1,13,232,3,9,13,16,16
4,5314,13,1,13,469,3,9,13,16,16


In [14]:
test = timeFeatures(test)
gc.collect()

299

In [15]:
out = pd.DataFrame()
out['click_id'] = test['click_id'].astype('int')
test.drop(['click_id'], axis=1, inplace=True)
test.head()

,ip,app,device,os,channel,wday,day,hour,minute,second
0,5744,9,1,3,107,4,10,4,0,0
1,119901,9,1,3,466,4,10,4,0,0
2,72287,21,1,19,128,4,10,4,0,0
3,78477,15,1,13,111,4,10,4,0,0
4,123080,12,1,13,328,4,10,4,0,0


### Ещё фичи

In [16]:
def features(train):    
    most_freq_hours_in_data    = [4, 5, 9, 10, 13, 14]
    middle1_freq_hours_in_data = [16, 17, 22]
    least_freq_hours_in_data   = [6, 11, 15]
    train['in_hh'] = (   4 
                     - 3*train['hour'].isin(  most_freq_hours_in_data ) 
                     - 2*train['hour'].isin(  middle1_freq_hours_in_data ) 
                     - 1*train['hour'].isin( least_freq_hours_in_data ) ).astype('uint8')

    gp    = train[['ip', 'day', 'in_hh', 'channel']].groupby(by=['ip', 'day', 'in_hh'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'nip_day_hh'})
    train = train.merge(gp, on=['ip','day','in_hh'], how='left')
    train.drop(['in_hh'], axis=1, inplace=True)
    train['nip_day_hh'] = train['nip_day_hh'].astype('uint32')
    del gp
    gc.collect()
    return train

In [17]:
# Define all the groupby transformations
GROUPBY_AGGREGATIONS = [
    # Variance in day, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'day', 'agg': 'var', 'type': 'float32'},
    # Variance in day, for ip-app-device
    {'groupby': ['ip','app','device'], 'select': 'day', 'agg': 'var', 'type': 'float32'},
    # Variance in day, for ip-app-os
    {'groupby': ['ip','app','os'], 'select': 'day', 'agg': 'var', 'type': 'float32'},
    
    # Variance in hour, for ip-app-channel
    #{'groupby': ['ip','app','channel'], 'select': 'hour', 'agg': 'var'},
    # Variance in hour, for ip-app-device
    #{'groupby': ['ip','app','device'], 'select': 'hour', 'agg': 'var'},
    # Variance in hour, for ip-app-os
    #{'groupby': ['ip','app','os'], 'select': 'hour', 'agg': 'var'},

    # Count, for ip-day
    #{'groupby': ['ip','day'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-day
    #{'groupby': ['ip','day'], 'select': 'device', 'agg': 'count'},
    # Count, for ip-day
    #{'groupby': ['ip','day'], 'select': 'os', 'agg': 'count'},
    
    # Count, for ip-hour
   # {'groupby': ['ip','hour'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-hour
    #{'groupby': ['ip','hour'], 'select': 'device', 'agg': 'count'},
    # Count, for ip-hour
    #{'groupby': ['ip','hour'], 'select': 'os', 'agg': 'count'},

    # Count, for ip-day-hour
    {'groupby': ['ip','day','hour'], 'select': 'channel', 'agg': 'count', 'type': 'uint32'},
    # Count, for ip-day-hour
    #{'groupby': ['ip','day','hour'], 'select': 'device', 'agg': 'count', 'type': 'uint32'},
    # Count, for ip-day-hour
   # {'groupby': ['ip','day','hour'], 'select': 'os', 'agg': 'count', 'type': 'uint32'},
    
    # Count, for ip-app
    {'groupby': ['ip', 'app'], 'select': 'channel', 'agg': 'count', 'type': 'uint32'},        
    # Count, for ip-app-os
    {'groupby': ['ip', 'app', 'os'], 'select': 'channel', 'agg': 'count', 'type': 'uint32'},
    # Count, for ip-app-day-hour
    {'groupby': ['ip','app','day','hour'], 'select': 'channel', 'agg': 'count', 'type': 'uint32'},
    
    # Mean hour, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'hour', 'agg': 'mean', 'type': 'float32', 'type': 'float32'}
]
def features2(train): 
# Apply all the groupby transformations
    for spec in GROUPBY_AGGREGATIONS:
        #print(f "Grouping by {spec['groupby']}, and aggregating {spec['select']} with {spec['agg']}")
    
        # Unique list of features to select
        all_features = list(set(spec['groupby'] + [spec['select']]))
        # Name of new feature
        new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), spec['agg'], spec['select'])
         # Perform the groupby
        gp = train[all_features]. \
            groupby(spec['groupby'])[spec['select']]. \
            agg(spec['agg']). \
            reset_index(). \
            rename(index=str, columns={spec['select']: new_feature}).astype(spec['type'])
         # Merge back to X_train
        train = train.merge(gp, on=spec['groupby'], how='left')
    del gp
    gc.collect()
    train['app']           = train['app'].astype('uint16')
    train['channel']       = train['channel'].astype('uint16')
    train['device']        = train['device'].astype('uint16')
    train['ip']            = train['ip'].astype('uint32')
    train['os']            = train['os'].astype('uint16')
    print("End")
    return train

In [18]:
train = features(train)
train = features2(train)
train.drop('ip', axis=1, inplace=True)
train.head()
cols = list(train.columns.values)

End


In [19]:
#print(train.dtypes)

In [20]:
test = features(test)
test = features2(test)
test.drop('ip', axis=1, inplace=True)
test.head()

End


,app,device,os,channel,wday,day,hour,minute,second,nip_day_hh,ip_app_channel_var_day,ip_app_device_var_day,ip_app_os_var_day,ip_day_hour_count_channel,ip_app_count_channel,ip_app_os_count_channel,ip_app_day_hour_count_channel,ip_app_channel_mean_hour
0,9,1,3,107,4,10,4,0,0,91,0.0,0.0,NaN,34,28,1,8,4.500000
1,9,1,3,466,4,10,4,0,0,2083,0.0,0.0,0.0,403,289,5,50,8.228572
2,21,1,19,128,4,10,4,0,0,2135,0.0,0.0,0.0,229,312,24,13,7.969697
3,15,1,13,111,4,10,4,0,0,1201,0.0,0.0,0.0,239,42,23,10,4.000000
4,12,1,13,328,4,10,4,0,0,208,0.0,0.0,0.0,60,24,7,7,4.000000


### Кодирование фичей

In [21]:
enc = ce.TargetEncoder(cols=['app', 'device', 'os', 'channel','wday', 'day','hour', 'minute', 'second']).fit(train, y)

In [22]:
#numeric_dataset = enc.transform(train)
train = enc.transform(train)
gc.collect()

454

In [23]:
#numeric_dataset.head()

In [24]:
#train.drop(['app', 'device', 'os', 'channel','wday', 'day','hour', 'minute', 'second'], axis=1, inplace=True)

In [25]:
train.head()

,nip_day_hh,ip_app_channel_var_day,ip_app_device_var_day,ip_app_os_var_day,ip_day_hour_count_channel,ip_app_count_channel,ip_app_os_count_channel,ip_app_day_hour_count_channel,ip_app_channel_mean_hour,app,device,os,channel,wday,day,hour,minute,second
0,9448,0.0,0.0,0.0,4924,864,20,470,13.264706,0.000721,0.001426,0.002040,0.000291,0.002438,0.002438,0.002549,0.002609,0.00264
1,42,NaN,0.0,NaN,14,4,1,2,13.000000,0.000080,0.001426,0.001431,0.000075,0.002438,0.002438,0.002549,0.002609,0.00264
2,2102,0.0,0.0,0.0,1089,491,96,253,13.462365,0.000080,0.001426,0.001159,0.000152,0.002438,0.002438,0.002549,0.002609,0.00264
3,381,0.0,0.0,0.0,215,65,16,33,13.571428,0.000721,0.001426,0.001159,0.000314,0.002438,0.002438,0.002549,0.002609,0.00264
4,55219,0.0,0.0,0.0,28826,1565,314,785,13.553073,0.000068,0.001426,0.001159,0.000232,0.002438,0.002438,0.002549,0.002609,0.00264


In [26]:
#train = pd.concat([numeric_dataset, train], axis=1, join_axes=[train.index])

In [27]:
#train = train.reindex_axis(cols, axis=1)
train = train.reindex(cols, axis=1)

In [28]:
train.head()

,app,device,os,channel,wday,day,hour,minute,second,nip_day_hh,ip_app_channel_var_day,ip_app_device_var_day,ip_app_os_var_day,ip_day_hour_count_channel,ip_app_count_channel,ip_app_os_count_channel,ip_app_day_hour_count_channel,ip_app_channel_mean_hour
0,0.000721,0.001426,0.002040,0.000291,0.002438,0.002438,0.002549,0.002609,0.00264,9448,0.0,0.0,0.0,4924,864,20,470,13.264706
1,0.000080,0.001426,0.001431,0.000075,0.002438,0.002438,0.002549,0.002609,0.00264,42,NaN,0.0,NaN,14,4,1,2,13.000000
2,0.000080,0.001426,0.001159,0.000152,0.002438,0.002438,0.002549,0.002609,0.00264,2102,0.0,0.0,0.0,1089,491,96,253,13.462365
3,0.000721,0.001426,0.001159,0.000314,0.002438,0.002438,0.002549,0.002609,0.00264,381,0.0,0.0,0.0,215,65,16,33,13.571428
4,0.000068,0.001426,0.001159,0.000232,0.002438,0.002438,0.002549,0.002609,0.00264,55219,0.0,0.0,0.0,28826,1565,314,785,13.553073


### Задаем параметры модели

In [29]:
params = {'eta': 0.3,
          'tree_method': "hist",
          'grow_policy': "lossguide",
          'max_leaves': 1400,  
          'max_depth': 0, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':0,
          'alpha':4,
          'objective': 'binary:logistic', 
          'scale_pos_weight':9,
          'eval_metric': 'auc', 
          'nthread':8,
          'random_state': 99, 
          'silent': True}

In [30]:
dtrain = xgb.DMatrix(train, y)#, feature_names=df_columns)
dtest = xgb.DMatrix(test)#, feature_names=df_columns)

In [31]:
del train, test
gc.collect()

512

In [32]:
n_folds = 5
early_stopping = 20
n_boost_round = 50


In [33]:
cv_output = xgb.cv(params, dtrain, num_boost_round = n_boost_round, nfold=n_folds, stratified = True,
                   early_stopping_rounds=early_stopping, verbose_eval=1)

[0]	train-auc:0.938731+0.000310231	test-auc:0.938699+0.00134723
[1]	train-auc:0.941342+0.00172917	test-auc:0.941286+0.00201851
[2]	train-auc:0.949392+0.00957539	test-auc:0.949329+0.00951941
[3]	train-auc:0.949529+0.00955319	test-auc:0.9494+0.00948943
[4]	train-auc:0.953301+0.00952959	test-auc:0.953138+0.00971441
[5]	train-auc:0.954255+0.0102466	test-auc:0.954156+0.0104185
[6]	train-auc:0.954822+0.00990272	test-auc:0.954471+0.0100302
[7]	train-auc:0.960016+0.00928037	test-auc:0.959815+0.00979661
[8]	train-auc:0.960557+0.0096099	test-auc:0.960202+0.00998335
[9]	train-auc:0.965828+0.00242699	test-auc:0.965316+0.0035097
[10]	train-auc:0.965919+0.00250668	test-auc:0.965385+0.00354807
[11]	train-auc:0.966763+0.00299741	test-auc:0.965853+0.00387067
[12]	train-auc:0.966896+0.00281027	test-auc:0.96595+0.00368733
[13]	train-auc:0.967118+0.0026584	test-auc:0.966074+0.00363544
[14]	train-auc:0.968448+0.00328554	test-auc:0.967444+0.00457306
[15]	train-auc:0.968691+0.00372463	test-auc:0.967852+0.004

In [34]:
num_boost_rounds = len(cv_output)
model = xgb.train(dict(params, silent=0), dtrain, num_boost_rounds)

In [35]:
#plot_importance(model)
#plt.gcf().savefig('feature_importance_xgb.png')

In [36]:
out['is_attributed'] = model.predict(dtest)
out.head()

,click_id,is_attributed
0,0,0.750804
1,1,0.727977
2,2,0.727977
3,3,0.727977
4,4,0.727977


In [37]:
out.to_csv('D:/ML/my8_xgb_sub_5m.csv', float_format='%.8f', index=False)